In [8]:
import pandas as pd
import scipy.stats as stats
from itertools import combinations
from statsmodels.stats.multitest import multipletests

In [4]:

file_path = 'result.csv'
data = pd.read_csv(file_path)

data.head()

,codec,hw_accel,power
0,H264,True,4.074817
1,H265,True,5.597625
2,AV1,False,13.960303
3,H264,True,4.046925
4,AV1,False,14.203015


In [5]:
# Shapiro-Wilk test for normality
normality_results = {}

groups = data.groupby(['codec', 'hw_accel'])

for group, group_data in groups:
    stat, p_value = stats.shapiro(group_data['power'])
    normality_results[group] = p_value

normality_results


{('AV1', False): 0.03352661430835724,
 ('H264', False): 0.8308635354042053,
 ('H264', True): 0.0012765637366101146,
 ('H265', False): 0.0004201561096124351,
 ('H265', True): 8.998152445371943e-09}

In [6]:
# Kruskal-Wallis H-test for independent samples
samples = [group_data['power'].values for group, group_data in groups]

stat, p_value = stats.kruskal(*samples)
stat, p_value


(131.15052862066284, 2.2097210863946276e-27)

In [9]:
group_names = list(groups.groups.keys())
comparisons = list(combinations(group_names, 2))

#Mann-Whitney U test for each pair and collect p-values
p_values = []
comparison_results = {}

for group1, group2 in comparisons:
    stat, p = stats.mannwhitneyu(groups.get_group(group1)['power'],
                                 groups.get_group(group2)['power'],
                                 alternative='two-sided')
    p_values.append(p)
    comparison_results[(group1, group2)] = {'stat': stat, 'p_value': p}

# Adjust p-values
adjusted_p_values = multipletests(p_values, method='bonferroni')[1]

for (group1, group2), p_value in zip(comparison_results.keys(), adjusted_p_values):
    comparison_results[(group1, group2)]['adjusted_p_value'] = p_value

comparison_results


{(('AV1', False), ('H264', False)): {'stat': 827.0,
  'p_value': 2.5263629217472197e-10,
  'adjusted_p_value': 2.52636292174722e-09},
 (('AV1', False), ('H264', True)): {'stat': 900.0,
  'p_value': 3.019859359162157e-11,
  'adjusted_p_value': 3.019859359162157e-10},
 (('AV1', False), ('H265', False)): {'stat': 819.0,
  'p_value': 5.092196223484032e-08,
  'adjusted_p_value': 5.092196223484032e-07},
 (('AV1', False), ('H265', True)): {'stat': 900.0,
  'p_value': 3.019859359162157e-11,
  'adjusted_p_value': 3.019859359162157e-10},
 (('H264', False), ('H264', True)): {'stat': 840.0,
  'p_value': 6.680905866173482e-11,
  'adjusted_p_value': 6.680905866173482e-10},
 (('H264', False), ('H265', False)): {'stat': 186.0,
  'p_value': 0.00027972315726719634,
  'adjusted_p_value': 0.0027972315726719635},
 (('H264', False), ('H265', True)): {'stat': 840.0,
  'p_value': 6.680905866173482e-11,
  'adjusted_p_value': 6.680905866173482e-10},
 (('H264', True), ('H265', False)): {'stat': 0.0,
  'p_value':